## Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive')

In [ ]:
%cd /content/MyDrive/MyDrive/PredNet_ApparentMotion

## Import

In [ ]:
import os
import numpy as np

import torch

from torch.autograd import Variable

from prednet_lotterly2 import PredNet

## Set Variables and Parameters

In [ ]:
batch_size = 1
A_channels = (3, 48, 96, 192)
R_channels = (3, 48, 96, 192)

In [ ]:
model = PredNet(R_channels, A_channels, output_mode='out_all')
model.load_state_dict(torch.load('./models/training.pt'                                 map_location=torch.device('cpu')))

if torch.cuda.is_available():
    print("Using GPU.")
    model.cuda()

## Load Test Files

In [ ]:
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

In [ ]:
# # KITTI test

# allinputs = []
# for i in [0,2,4,6,8]:
#     inputfn = './KITTI/{:010d}.npy'.format(i))
#     inputarray = np.load(inputfn).transpose(2,0,1).tolist()
#     allinputs.append(inputarray)
# allinputs = np.array([allinputs])
# print(allinputs.shape)

# inputs = torch.from_numpy(allinputs)
# inputs = Variable(inputs)
# nt = inputs.shape[1]

# ALL = model(inputs)

In [ ]:
# DOT test

inputfn = './Set9/Original/Set9.npy'
inputarray = np.load(inputfn)

inputs = inputarray[0]
inputs = np.transpose(inputs,(0,3,1,2))
inputs = np.array([inputs])

inputs = torch.from_numpy(inputs)
inputs = Variable(inputs)
nt = inputs.shape[1]

ALL = model(inputs)

## Save All of Layers as Numpy File

In [ ]:
namelist = ["A", "Ahat", "E"]
for i in range(len(ALL)-1):
    for t in range(5):
        for l in range(4):
            org = ALL[i]
            orgLayer = org[t*4+l][0]
            fn = "{}{}_t{}.npy".format(namelist[i],l,t)
            orgdir = "./results/dot/{}".format(fn)
            np.save(orgdir, orgLayer)
R = ALL[len(ALL)-1]
for t in range(5):
    for l in range(4):
        orgR = R[t*4+l][0]
        fn = "R{}_t{}.npy".format((3-l),t)
        Rdir = "./results/dot/{}".format(fn)
        np.save(Rdir,orgR)